In [ ]:
import requests
import pandas as pd
import json

from typing import Dict
from pandas.io.json import json_normalize

In [ ]:
def get_structure_course(url: str) -> Dict[str, str]:
    """Function gets cource structure from API"""

    try:
        r = requests.post(url)
        r.raise_for_status()
    except requests.exceptions.HTTPError as errh:
        print('HTTPError: {}'.format(errh))
        print(r.content.decode('UTF-8'))
        raise
    
    result = json.loads(r.text)
    
    return result

In [ ]:
url = 'http://84.201.129.203:4545/get_structure_course'
result = get_structure_course(url)

In [ ]:
with open('json_structure.json', 'w') as f:
    json.dump(result, f)

In [ ]:
with open('json_structure.json', 'r') as f:
    data = json.load(f)

In [ ]:
children_id = set()
for block in data['blocks']:
    children_id.update(data['blocks'][block].get('children', []))

In [ ]:
node_id = dict()
for block in data['blocks']:
    if data['blocks'][block]['id'] in children_id:
        node_id[data['blocks'][block]['id']] = data['blocks'][block]['display_name']
    else:
        root_id = data['blocks'][block]['id']

In [ ]:
def print_structure(data, root_id, result={}, output='\t'):
    if data['blocks'][root_id].get('children', 0) == 0:
        return
    else:
        output_string = output + str(data['blocks'][root_id]['display_name'])+': '+str(data['blocks'][root_id]['id'])
        print(output_string)
        result[data['blocks'][root_id]['display_name']] = data['blocks'][root_id]['id']
        for child_id in data['blocks'][root_id]['children']:
            print_structure(data, child_id, result, '\t' + output)
    return result

In [ ]:
res = print_structure(data, root_id)

In [ ]:
df = pd.DataFrame(res.items(), columns = ['display_name', 'id'])

In [ ]:
df